<a href="https://colab.research.google.com/github/lxknvlk/google-collab/blob/main/FAST_STABLE_DIFFUSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copied from: https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb

In [1]:
#@markdown #Mount Drive
from google.colab import drive

mount_drive = True #@param {type: "boolean"}

if mount_drive:
  drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@markdown #Install/Update AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [3]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf gcolab.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

clear_output()
inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [4]:
#@markdown # Install ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse

Model = "All"

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors'
Depth='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors'
HED='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors'
MLSD='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors'
Normal='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors'
OpenPose='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors'
Scribble='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors'
Seg='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors'


with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd /content
  else:
    %cd sd-webui-controlnet
    !git pull
    %cd /content

!cp /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models/*.yaml /content/gdrive/MyDrive/sd/stable-diffusion-webui/models
mdldir="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models"

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt
with open("CN_models.txt", 'r') as f:
  mdllnk = f.read().splitlines()
!rm CN_models.txt

if Model == "All": 
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

elif Model == "T2iadapter_Models":
  mdllnk=list(filter(lambda x: 't2i' in x, mdllnk))
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

else:
  download(globals()[Model], mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [5]:
#@markdown # Install Extensions

from git import Repo

def install(url, name):
  DL_PATH = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/"+name
  Repo.clone_from(url, DL_PATH)

  if os.path.exists(f'{DL_PATH}'):    
    pass
  else:
    error = True

clear_output()

error = False

install("https://github.com/kohya-ss/sd-webui-additional-networks.git", "sd-webui-additional-networks")
install("https://github.com/fkunn1326/openpose-editor.git", "openpose-editor")
#install("https://github.com/d8ahazard/sd_dreambooth_extension.git", "sd_dreambooth_extension")

if error == False:
  clear_output()
  inf('\u2714 All extensions copied and verified.','success', '350px')
else:
  inf('\u2718 Error','danger', "300px")


    


Button(button_style='success', description='✔ All extensions copied and verified.', disabled=True, layout=Layo…

In [ ]:
#@markdown # Download Models from network

import gdown

clear_output()

dlpath="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/"

error = False

def download(url, name):
  fullpath = dlpath+name

  gdown.download(url, fullpath, quiet=False)
  if os.path.exists(f'{fullpath}'):    
    pass
  else:
    error = True

download("https://civitai.com/api/download/models/13128", "clarity.safetensors")
#download("https://civitai.com/api/download/models/21216", "babes.safetensors")
#download("https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt", "v1-5-pruned.ckpt")



#download("https://civitai.com/api/download/models/19590", "realbiter.safetensors")
#download("https://civitai.com/api/download/models/17238", "hyperreal.safetensors")

#download("https://civitai.com/api/download/models/2409", "tentacles")
#download("https://civitai.com/api/download/models/11887", "cyberpunk")


download("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5.ckpt", "anything-v4.5.ckpt")

#download("URL", "NAME")

if error == False:
  clear_output()
  inf('\u2714 All models copied and verified.','success', '350px')
else:
  inf('\u2718 Error','danger', "300px")


    


In [7]:
#@markdown # Download Models from my drive

import gdown
import shutil
import glob

clear_output()
DST="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/"
SRC = "/content/drive/MyDrive/Files/Models/*"

FILTER = "clarity" #@param {type:"string"}

for file in glob.glob(SRC):
  if FILTER in file:
    print(file)
    shutil.copy(file, DST)




    


/content/drive/MyDrive/Files/Models/clarity_19.safetensors


In [8]:
#@markdown # Download LORAS from my drive

import gdown
import shutil
import glob

clear_output()

DST ="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora/"
SRC = "/content/drive/MyDrive/Files/LORA/*"

for file in glob.glob(SRC):
    print(file)
    shutil.copy(file, DST)




    


/content/drive/MyDrive/Files/LORA/cameltoe_v21.safetensors
/content/drive/MyDrive/Files/LORA/3dInspiredLora_v10.safetensors
/content/drive/MyDrive/Files/LORA/sunsetrider.safetensors


In [ ]:
#@markdown # Start Stable-Diffusion
from IPython.utils import capture
import time
import sys
import fileinput

Use_localtunnel = False 

User = "" 
Password= "" 

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.9/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.9/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers $auth --disable-console-progressbars

[AddNet] Updating model hashes...
100% 3/3 [00:00<00:00, 11.33it/s]
[AddNet] Updating model hashes...
100% 3/3 [00:00<00:00, 5501.93it/s]
Calculating sha256 for /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5.ckpt: fbcf965a62d9d82e935d3d17e97522c29f44550aa9e120a6886f19b578521ec5
Loading weights [fbcf965a62] from /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5.ckpt
Creating model from config: /content/gdrive/MyDrive/sd/stable-diffusion-webui/configs/v1-inference.yaml
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Applying xformers cross attention optimization.
Textual inversion embeddings loaded(0): 
Model loaded in 37.5s (calculate hash: 22.8s, load weights from disk: 8.8s, create model: 2.3s, apply weights to model: 2.4s, apply half(): 0.5s, move model to device: 0.7s).
Running on public URL: https://e08bd5a2-acfe-487d.gradio.live
✔ Connected
Startup time: 71.2s (impo

In [ ]:
#@markdown # Copy output folder to my drive

FOLDER = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/outputs/txt2img-images/"
OUT = "/content/drive/MyDrive/Files/output"

from distutils.dir_util import copy_tree
copy_tree(FOLDER, OUT)




In [ ]:
#@markdown # Clean output folder in webui

FOLDER = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/outputs/"

import shutil


if os.path.exists(f'{FOLDER}'):    
  shutil.rmtree(FOLDER)

if os.path.exists(f'{FOLDER}'):    
  inf('\u2718 Error','danger', "300px")
else:
  inf('\u2714 Cleared outputs!.','success', '350px')

